In [ ]:
from tqdm import tqdm
import random
from random import sample
import networkx as nx
import re, logging
import openai, datetime, os
openai.api_key = ""
import json
def load_openai_keys():
    keys = []
    with open('../openai_keys_filter.txt', "r") as f:
        for line in f:
            key = line.strip().split()
            keys.append(key[-1])
    return keys
openai_api_keys = load_openai_keys()
random.shuffle(openai_api_keys)
def update_key():
    curr_key = openai_api_keys[0]
    openai.api_key = curr_key
    openai_api_keys.remove(curr_key)
    openai_api_keys.append(curr_key)

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    # Remove any existing handlers
    for handler in logger.handlers:
        logger.removeHandler(handler)
    # Output to file
    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    # # Output to terminal
    # sh = logging.StreamHandler()
    # sh.setFormatter(formatter)
    # logger.addHandler(sh)

    return logger

# multiple-list save to logging
def list2str(l):
    s = ''
    for i in l:
        s += str(i) + '\t'
    return s
def list_equal(a, answers):
    # if set(a) == set(b):
    #     return True
    # else:
    #     return False
    for b in answers:
        # if set(a) contain set(b)
        if set(b) == set(a):
            return True
        # if set(a) == set(b):
        #     return True
    return False
def dict2str(d):
    s = ''
    for k in d:
        s += str(k) + '\t' + str(d[k]) + '\r'
    return s

In [ ]:
# read jsonl file
def read_jsonl(file):
    data = []
    with open(file, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data
raw_data = read_jsonl('../OWA/depth-1/meta-test_symbolic.jsonl')

In [ ]:
filter_data = []
for data in raw_data:
    fid = 1
    lid = 1
    facts = ''
    rules = ''
    
    for tri_name in data['triples'].keys():
        tri = data['triples'][tri_name]
        # facts += 'F' + str(fid) + ': ' + tri['sym_text'][:-2] + '.\n'
        facts += tri['text'] + '\n'

        fid += 1
    for rule_name in data['rules'].keys():
        rule = data['rules'][rule_name]
        # rules += 'L' + str(lid) + ': ' + rule['sym_text'][:-2] + '.\n'
        rules += rule['text'] + '\n'

        lid += 1
    for q_name in data['questions'].keys():
        question = data['questions'][q_name]
        if question['QDep'] >= 1:
            if question['answer'] == True or question['answer'] == False:
                item = dict()
                item['question'] = question['question'] 
                item['answer'] = question['answer']
                item['facts'] = facts
                item['rules'] = rules
                filter_data.append(item)
            


In [ ]:
print(filter_data[2]['facts'])

In [ ]:
# # read few_shot_prompts.txt to get the prompts

# def read_few_shot_prompts(path):
#     prompts = list()
#     prompts_plus = list()
#     with open(path, 'r') as f:
#         blocks = f.read().split('\n\n')
#         for block in blocks:
#             # 正则表达式提取statement以后，answer之前的内容
#             question, answer = block.split('Reasoning: ')

            
#             d = {}
#             d['Statement'] = question
#             # answer = re.findall(r'Answer: (.*)', block, flags=re.DOTALL)[0]
#             d['Answer'] = "Reasoning: " + answer

#             d_plus = {}
#             d_plus['Statement'] = question
#             d_plus['Answer'] = "Reasoning: Let's think step by step. " + answer
            
#             prompts.append(d)
            
#     print(prompts)
#     return prompts, prompts_plus
# prompts, prompts_plus = read_few_shot_prompts('few_shot_prompts.txt')
# len(prompts)

In [ ]:
# read data
nowTime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
dir = 'logs/natural_standard_f+r'
if not os.path.exists(dir):
        os.makedirs(dir)
logger = get_logger(os.path.join(dir, nowTime + '.log'), verbosity=1)
# model = 'gpt-4'
model = "gpt-3.5-turbo"

logging.info('model: ' + model)



num = 0
false_num = 0
true_num = 0

pos_true = 0
pos_false = 0
neg_true = 0
neg_false = 0

for item in tqdm(filter_data):
    message = {

        'system': "You are a helpful assistant with abductive reasoning abilities. Given a set of rules and facts, you have to reason whether a statement is true or false. ",
        'user': "Given a set of rules and facts, you have to reason whether a statement is true or false. Here are some facts and rules: \n" + item['facts'] + item['rules'] + "Does it imply that the statement \"" + item['question'] + "\" is True?\nAnswer with only True or False. The answer is: ",
            

    }

    server_error_cnt = 0

    while server_error_cnt<10:
        try:
            
            update_key()
            
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                        {"role": "system", "content": message['system']},
                        {"role": "user", "content": message['user']},
                    ],
                temperature=0,

            )
            break
        except Exception as e:
            server_error_cnt += 1
            print(e)

    

    logger.info('message: \n' + dict2str(message)) 
        

    results = response['choices'][0]['message']['content']

    
    num += 1
    ans = results.split('\n')[0]
    if item['answer'] == True:
        if 'True' in ans:
            true_num += 1
            pos_true += 1
            logger.info('correctness: ' + 'Correct')
        elif 'False' in ans:
            false_num += 1
            pos_false += 1
            logger.info('correctness: ' + 'Incorrect')
        else:
            false_num += 1
            pos_false += 1
            logger.info(results)
    else:
        if 'True' in ans :
            false_num += 1
            neg_false += 1
            logger.info('correctness: ' + 'Incorrect')
        elif 'False' in ans:
            true_num += 1
            neg_true += 1
            logger.info('correctness: ' + 'Correct')
        else:
            false_num += 1
            pos_false += 1
            logger.info(results)

    logger.info('grounding truth: ' + str(item['answer']) + '\tprediction: ' + results)
    

        

logger.info("accuracy: " + str( true_num / num ))